# Case study of uniaxial tensile tests

This is an example of how to use ParaMaterial to process a dataset of uniaxial tensile test measurements.



## Data processing
- **Trim data.**
- **Extract material properties**
- **Make representative curves.**

Import libraries and objects.

In [1]:
import importlib
import os
import shutil

import matplotlib.pyplot as plt

import paramaterial as pam